In [26]:
# %cd ..
!nvidia-smi




Mon Mar 30 13:23:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 5596 (delta 19), reused 21 (delta 9), pack-reused 5557
Receiving objects: 100% (5596/5596), 13.35 MiB | 6.80 MiB/s, done.
Resolving deltas: 100% (3745/3745), done.


In [0]:
!pip install --upgrade keras
%cd keras-retinanet/

!pip install .
!python setup.py build_ext --inplace
!pip install gdown
!pip install tensorflow-gpu

     |████████████████████████████████| 378kB 1.4MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170478 sha256=575510631a0789348a894d9272036db6deae8301d36ebc742fe71b7c03798c83
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=e812a1a5f11cdadfb9bcf2f6a012f91aeebd7e7200497b735014850b8960e421
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/C

In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil

import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


In [0]:
annot_path = '/content/drive/My Drive/PersonDetection/WiderPerson/Annotations'
annot = os.listdir(annot_path)
images_path = '/content/drive/My Drive/PersonDetection/WiderPerson/Images'
annotations = 'annotations.csv'
#Remove classes which shall be left out in the csv
classes_ids = {1:'pedestrian',
               2:'riders',
               3:'partially-visible',
               5:'crowd'}


In [0]:
# Generate the classes csv file
with open(annotations, newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for filename in os.listdir(images_path)[0:10]:
    if str(filename + '.txt') in annot:
      f = open(annot_path + '/' +  filename + '.txt', 'r')
      
      for index, line in enumerate(f): 
        if index == 0: 
          if line.strip() == '0':
            print('Not any  object in the image!')
          continue
          
        else: 
          split_line = line[:line.find('/')].split(' ')
          first_char = split_line.pop(0)
          split_line.insert(len(split_line), first_char)
          split_line.insert(0, images_path + '/' + filename)
          #convert from index to class label 
          try:
            split_line[-1] = classes_ids[int(split_line[-1])]
          except: 
            continue
          split_line[0] = split_line[0]

          csv_writer.writerow(split_line)
      
      f.close()

In [0]:
#Check for wrongly annotaded bounding boxes
path = '/content/drive/My Drive/PersonDetection/keras-retinanet/annotations.csv'
def check_bb(path):
  colnames = ['filename', 'x1', 'y1', 'x2', 'y2', 'class_label']
  df = pd.read_csv(path, names=colnames)
  df_new = df.loc[(df.x1 < df.x2) & (df.y1 < df.y2), : ]
  print ('Reduces shape from ', df.shape, 'to ', df_new.shape)
  return df_new

df = check_bb(path)

Reduces shape from  (269826, 6) to  (269823, 6)


In [0]:
df.head()
df.to_csv('annotations_cleaned.csv', header=None, index=None)

In [0]:
#create indexing csv file
with open('classes.csv', newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for index, key in enumerate(classes_ids.keys()):
    csv_writer.writerow([str(classes_ids[key]), index])



# Training



In [0]:
#TODO: Make it also work for bigger architecture:"/fizyr/keras-retinanet/releases/download/0.5.1/resnet152_oid_v1.0.0.h5"

In [0]:
!ls
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

annotations.csv  CONTRIBUTORS.md  keras_retinanet  requirements.txt  snapshots
build		 examples	  LICENSE	   setup.cfg	     tests
classes.csv	 images		  README.md	   setup.py
Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [0]:
!python keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 500 --epochs 10 csv annotations.csv classes.csv

Using TensorFlow backend.
2020-03-30 16:11:00.658789: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-03-30 16:11:00.658920: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-03-30 16:11:00.658942: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Creating model, this may take a second...
2020-03-30 16:11:01.856486: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORC

In [0]:
shutil.("/content/drive/My Drive/PersonDetection/classes.csv", "/content/drive/My Drive/PersonDetection/keras-retinanet")


In [0]:
model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv(CLASSES_FILE, header=None).T.loc[0].to_dict()

# Testing


In [0]:
# Convert to inference Model 

In [0]:
def show_image_objects(image_rows):
  #TODO: make multiple boxes possible
  img_path_plot = images_path + '/' + image_row.iloc.image_name

  box = [
    image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
  ]

  image = read_image_bgr(img_path_plot)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_box(draw, box, color=(255, 255, 0))

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [0]:
classes_df = pd.read_csv('classes.csv')
show_image_objects(classes_df.iloc[0])